## NEXRAD Data Download and Analysis
Karl Schneider (kps5442@psu.edu)

In [1]:
import tempfile
from datetime import datetime, timedelta
import nexradaws


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



#### Data Download
An example of downloading NEXRAD Level 2 radar data for a particular time of interest using the nexradaws python module.  
https://nexradaws.readthedocs.io/en/latest/

In [2]:
# Download test data - look for files in window of 10 min before time of interest and grab most recent file
conn = nexradaws.NexradAwsInterface()
site = 'KCCX'
time = datetime(2017, 3, 3, 18, 30)
scans = conn.get_avail_scans_in_range(time-timedelta(minutes=10), time, site)

In [3]:
scans

[<AwsNexradFile object - 2017/03/03/KCCX/KCCX20170303_182436_V06>]

In [4]:
# Download data to a temporary directory
tmpdir = tempfile.mkdtemp()
download = conn.download(scans[-1], tmpdir)

Downloaded KCCX20170303_182436_V06
1 out of 1 files downloaded...0 errors


In [5]:
# Show the downloaded file
download.success

[<LocalNexradFile object - /tmp/tmpwuj69u0j/KCCX20170303_182436_V06>]

----